In [ ]:
Description:
This script reads the VGP Ordinal Phase1+ table and cleans it up for import into the GoaT database.
The script reads the table from a google spreadsheet link,
cleans up the table, and adds a project and status column to the table to conform to GoaT import.


In [3]:
import pandas as pd
import numpy as np

# Google Spreadsheet link:
# https://docs.google.com/spreadsheets/d/1vsV7OTU-BAeOkBSrsESGCHaGuLcFW6U9mUluy6II0tY/edit?gid=0#gid=0
#Download link:
tsv_link = "https://docs.google.com/spreadsheets/d/1Jwjv6Kwc6VIn1UMMhnG6kvFCxjwGdC5b7p_HtbDOMOs/export?format=tsv&id=1Jwjv6Kwc6VIn1UMMhnG6kvFCxjwGdC5b7p_HtbDOMOs&gid=1380659438"



In [4]:
# Select colums to import
columns = [
    "Order",
    "Lineage",
    "Superorder",
    "Family Scientific Name",
    "Scientific Name",
    "English Name",
    "NCBI taxon ID",
    "Status",
    "QV",
    "IUCN (2016-2024)",
    "CITES",
    "Main project",
    "Second project",
    "Publication"
]
# Read the table from the link
vgp_df = pd.read_csv(tsv_link,
                    delimiter="\t",
                    dtype=object,
                    usecols=columns
                    )

print('Vgp file successfuly opened. Starting cleanup...')
vgp_df.head()


Vgp file successfuly opened. Starting cleanup...


,Order,Lineage,Superorder,Family Scientific Name,Scientific Name,English Name,NCBI taxon ID,Status,QV,IUCN (2016-2024),CITES,Main project,Second project,Publication
0,m1,Mammals,Afrotheria,Elephantidae,Elephas maximus indicus,Asian elephant,99487,4,NaN,Endangered,NaN,VGP,Collosal,NaN
1,m1,Mammals,Afrotheria,Elephantidae,Loxodonta africana,African elephant,9785,NaN,NaN,Endangered,NaN,VGP,Collosal,NaN
2,m2,Mammals,Afrotheria,Procaviidae,Heterohyrax brucei,yellow-spotted rock hyrax,77598,4,NaN,Least Concern,NaN,VGP,NaN,NaN
3,m3,Mammals,Afrotheria,Dugongidae,Dugong dugon,dugong,29137,4,NaN,Endangered,NaN,VGP,NaN,NaN
4,m3,Mammals,Afrotheria,Trichechidae,Trichechus inunguis,Amazon manatee,9777,NaN,NaN,Vulnerable,NaN,Amazoomics,NaN,NaN


In [2]:
def vgp_table_cleanup(df):
    """
    Cleans up a pandas DataFrame by performing the following actions:
    - Replaces empty or whitespace-only strings with NaN.
    - Strips leading and trailing spaces from all string values.
    - Drops columns and rows where all values are NaN.
    
    Args:
        df (pandas.DataFrame): The input DataFrame to be cleaned.

    Returns:
        pandas.DataFrame: The cleaned DataFrame.
    """
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df.replace(r"^ +| +$", r"", regex=True)
    df.dropna(how="all", axis=1, inplace=True)
    df.dropna(how="all", axis=0, inplace=True)
    return df

def cleanup_vgp_headers_specific_units(df):
    """
    Cleans up the headers of a VGP table by performing the following actions:
    - Replaces spaces with underscores.
    - Converts all characters to lowercase.
    - Removes parentheses.

    Args:

        df (pandas.DataFrame): The input DataFrame to be cleaned.

    Returns:
        pandas.DataFrame: The cleaned DataFrame.
    """
    df.columns = (
        df.columns
        .str.replace(' ', '_')
        .str.replace(r'\(', '',regex=True)
        .str.replace(r'\)', '',regex=True)
        .str.lower()
    )
    return df



In [5]:
# Clean up the table:
vgp_df = cleanup_vgp_headers_specific_units(vgp_table_cleanup(vgp_df))

print('Vgp file successfuly cleaned. Treating project columns...')

# Add a project column to the table
vgp_df["project"] = "VGP"

# Create a column with all projects working on the species

vgp_df['all_projects'] = vgp_df.apply(
    lambda row: ','.join(
        sorted(set(x for x in [row['project'], row['main_project'], row['second_project']] if pd.notna(x)))
    ),
    axis=1
)


# Translate the project names to acronyms when they are valid projects
# (needs manual update if new projects are added)
translate_to_acronyms = {
                            'Sanger 25G,VGP':'25GP,VGP',
                            'AfricaBP,VGP': 'AFRICABP,VGP', 
                            'Cetacean GP,VGP': 'CGP,VGP',
                            'VGP,Yggdrasil': 'VGP,YGG',
                            'CatalanBP,VGP': 'CBP,VGP',
                            'Canadian Biogenome Project,VGP': 'CANBP,VGP',
                            'Threatened Species Initiative (TSI),VGP': 'TSI,VGP',
                            'Canada Biogenome Project,VGP': 'CANBP,VGP',
                            'Minderoo OceanOmics,VGP': 'OG,VGP',
                            'Sanger 25G project,VGP': '25GP,VGP'
                        }

# Check for projects that need acronyms added using the following code
print(vgp_df['all_projects'].unique())
print('chech if any above should be translated to project acronym. Expanding status columns...')


# Map the acronyms to the project names in the all_projects column if they are in the translate_to_acronyms dictionary but leave the rest as is
vgp_df['all_projects'] = vgp_df['all_projects'].apply(
    lambda i: translate_to_acronyms[i] if i in translate_to_acronyms.keys() else i
)

possible_seq_status = ["sample_collected","sample_acquired","in_progress","data_generation","in_assembly","insdc_submitted","open","insdc_open","published"]
for item in possible_seq_status:
    if item not in vgp_df:
        vgp_df[item] = np.nan


# Map the status to the GoaT status
status_to_map = {
                '0': "",
                '1': "sample_collected",
                '2': "",
                '3': "in_progress",
                '4': "open",
                '5': "open",
                }

vgp_df['sequencing_status'] = vgp_df['status'].map(status_to_map)
# Map existing status for specific project combination to each status columns
for item in possible_seq_status:
    vgp_df.loc[vgp_df['sequencing_status'] == item, item] = vgp_df['all_projects']

# Populate the status columns with the project names using the hierarchy of the status
vgp_df.loc[vgp_df["published"] == vgp_df['all_projects'], "insdc_open"] = vgp_df['all_projects']
vgp_df.loc[vgp_df['insdc_open'] == vgp_df['all_projects'], 'open'] = vgp_df['all_projects']
vgp_df.loc[vgp_df['open'] == vgp_df['all_projects'], 'in_progress'] = vgp_df['all_projects']
vgp_df.loc[vgp_df['data_generation'] == vgp_df['all_projects'], 'in_progress'] = vgp_df['all_projects']
vgp_df.loc[vgp_df['in_assembly'] == vgp_df['all_projects'], 'in_progress'] = vgp_df['all_projects']
vgp_df.loc[vgp_df['in_progress'] == vgp_df['all_projects'], 'sample_acquired'] = vgp_df['all_projects']
vgp_df.loc[vgp_df['sample_acquired'] == vgp_df['all_projects'], 'sample_collected'] = vgp_df['all_projects']

print("Generating VGP_Ordinal_Phase1_plus.tsv file...")
vgp_df.to_csv("VGP_Ordinal_Phase1_plus.tsv",sep="\t", index=False)

Vgp file successfuly cleaned. Treating project columns...
['Collosal,VGP' 'VGP' 'Amazoomics,VGP' 'BRAIN,VGP' 'HPRC,VGP' 'T2T,VGP'
 'Allen Institute,VGP' 'Sanger 25G,VGP' 'DToL,VGP' 'Cohen Lab,VGP'
 'ERGA,VGP' 'CCGP,VGP' 'Revive & Restore,VGP' 'AfricaBP,VGP'
 'Cetacean GP,VGP' 'Cetacean GP,DToL,VGP' 'Individual,VGP'
 'Narwhal project,VGP' 'Cattle,VGP' 'VGP,Yggdrasil'
 'Individual contribution, Ruminant T2T,VGP' 'Qatar,VGP'
 'Bat1K,Paratus,VGP' 'Bat1K,VGP' 'COVID-19,VGP' 'Invidual,VGP'
 'AmaZoomics,VGP' 'Colossal,VGP' 'Allen,VGP' 'DToL?,VGP'
 'VGP,Vocal learning' 'Brood Parasitic,VGP' 'RCN Manakin,VGP' 'Falcon,VGP'
 'B10K,VGP' 'CatalanBP,VGP' 'Self,VGP' 'DToL, ERGA,VGP'
 'Canadian Biogenome Project,VGP'
 'Threatened Species Initiative (TSI),VGP' 'VGP,hypothalamus'
 'Hypothalamus,VGP' 'Canada Biogenome Project,VGP'
 'Minderoo OceanOmics,VGP' 'Sanger 25G project,VGP']
chech if any above should be translated to project acronym. Expanding status columns...
Generating VGP_Ordinal_Phase1_plus.

C:\Users\fchen13\AppData\Local\Temp\ipykernel_9980\743129591.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)
C:\Users\fchen13\AppData\Local\Temp\ipykernel_9980\2751846999.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'AFRICABP,VGP' 'VGP'
 'AFRICABP,VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'AFRICABP,VGP'
 'CCGP,VGP' 'CCGP,VGP' 'VGP' 'VGP' 'VGP' 'Hypothalamus,VGP' 'VGP' 'VGP'
 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP' 'VGP'
 'VGP' 'VGP' 'VGP']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  vgp_df.loc[vgp_d